In [1]:
from workflows import DYStudiesProcessor

import json
from coffea.processor import (
    run_uproot_job,
    iterative_executor, 
    futures_executor,
    NanoAODSchema
)

with open("dummy_samples.json") as f:
    samples = json.load(f)

import json
import os.path as osp
metaconditions_file = "metaconditions/Era2017_legacy_v1.json"
with open(metaconditions_file) as f:
    metaconditions = json.load(f)    

dystudies = DYStudiesProcessor(metaconditions, False, True, './outputs/')

histos = run_uproot_job(
    samples,
    'Events',
    dystudies,
    executor=futures_executor,
    executor_args={"schema": NanoAODSchema,"workers": 4},
    )




In [ ]:
x = ak.ones_like(ak.flatten(events.Photon.pt))

In [ ]:
import pandas as pd
import pyarrow as pa
import awkward as ak

x = pd.read_parquet('./outputs/test')

y = ak.from_arrow(pa.Table.from_pandas(x))

In [ ]:
# di-photon MVA
import xgboost as xg

min_diphoton_mass = 100
max_diphoton_mass = 180

model = xg.Booster()
model.load_model('aux-data/altDiphoModel_coffea.model')

# get the number of diphotons per row
# and save for re-wrapping xgb outputs
counts = ak.num(diphotons, axis=1)

# extract diphoton vars into flat lists
dipho_leadIDMVA = ak.flatten(diphotons["0"].mvaID)
dipho_subleadIDMVA = ak.flatten(diphotons["1"].mvaID)
dipho_lead_ptoM = ak.flatten(diphotons["0"].pt / diphotons.mass)
dipho_sublead_ptoM = ak.flatten(diphotons["1"].pt / diphotons.mass)
dipho_lead_eta = ak.flatten(diphotons["0"].eta)
dipho_sublead_eta = ak.flatten(diphotons["1"].eta)

diphoVars  = ['dipho_leadIDMVA', 'dipho_subleadIDMVA', 'dipho_lead_ptoM', 
              'dipho_sublead_ptoM', 'dipho_leadEta', 'dipho_subleadEta', 
              'CosPhi', 'vtxprob', 'sigmarv', 'sigmawv']
allVars = diphoVars + ["dipho_mass"]

f = uproot.open('data/ggH_powheg_UL_2017.root')
tree = f['vbfTagDumper/trees/ggh_125_13TeV_GeneralDipho']
arrays = tree.arrays(allVars, how=dict)

mask = (  (arrays["dipho_mass"]> min_diphoton_mass) & (arrays["dipho_mass"]< max_diphoton_mass) 
        & (arrays["dipho_leadIDMVA"]>-0.9) & (arrays["dipho_subleadIDMVA"]>-0.9) 
        & (arrays["dipho_lead_ptoM"]>0.333) & (arrays["dipho_sublead_ptoM"]>0.25))

x = np.column_stack((ak.to_numpy(arrays[var][mask]) for var in diphoVars))[:100]

print(x.shape)

diphoMatrix = xg.DMatrix(x, feature_names=diphoVars)


y = model.predict(diphoMatrix)

In [ ]:
import uproot
import xgboost
import numpy as np
import awkward as ak

diphoVars  = ['dipho_leadIDMVA', 'dipho_subleadIDMVA', 'dipho_lead_ptoM', 
              'dipho_sublead_ptoM', 'dipho_leadEta', 'dipho_subleadEta', 
              'CosPhi', 'vtxprob', 'sigmarv', 'sigmawv']
allVars = diphoVars + ["dipho_mass"]

f = uproot.open('../data/ggH_powheg_UL_2017.root')
tree = f['vbfTagDumper/trees/ggh_125_13TeV_GeneralDipho']
arrays = tree.arrays(allVars, how=dict)

mask = (  (arrays["dipho_mass"]>100.) & (arrays["dipho_mass"]<180.) 
        & (arrays["dipho_leadIDMVA"]>-0.9) & (arrays["dipho_subleadIDMVA"]>-0.9) 
        & (arrays["dipho_lead_ptoM"]>0.333) & (arrays["dipho_sublead_ptoM"]>0.25))

x = np.column_stack((ak.to_numpy(arrays[var][mask]) for var in diphoVars))[:100]

print(x.shape)

diphoMatrix = xgboost.DMatrix(x, feature_names=diphoVars)

model = xgboost.Booster()
model.load_model('aux-data/altDiphoModel_coffea.model')
y = model.predict(diphoMatrix)

print(x)
print(y)

In [ ]:
events.behavior["__events_factory__"]._partition_key.replace('/', '_') + '.parquet'

In [ ]:
events.metadata

In [ ]:
metaconditions

In [ ]:
from workflows import taggers

In [ ]:
dir(taggers)